In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
import nltk
import requests
import random
import math
import pathlib 

from modules.create_path import create_path
from modules.hein_scraping_functions import create_browser, webpage_wait, get_paper_data, mod_names, check_google, similar_names, search_names
from modules.data_manipulation_functions import remove_commas, check_files


In [2]:

# Create the paths for the data directories
input_path, work_path, intr_path, out_path, selenium_driver_path = create_path()

# Create the paths for the Chrome binary and selenium driver
chrome_binary_path = pathlib.Path("C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe")
selenium_driver_full_path = selenium_driver_path / "chromedriver.exe"

# Initalize the browsers that we are going to use
driver = create_browser(chrome_binary_path, selenium_driver_full_path)
g_driver = create_browser(chrome_binary_path, selenium_driver_full_path)

driver.get("http://proxy.its.virginia.edu/login?url=http://heinonline.org/HOL/Welcome")

In [5]:
# List the files that are in the intermediate directory
# This is where we will be storing the individual output files
#These three lines give a list of the files that have already been scraped
#Papers gives a list of the papers by each author
#Skip gives a list of the skipped names
current_data = os.listdir(intr_path)

# Load the datasets from the working directory
# The datasets in the working directory have already 
# been cleaned.
control = pd.read_excel(work_path / "control.xlsx")
lateral = pd.read_excel(work_path / "lateral_test.xlsx")

# Load the name modification dataset
# This is a dataframe of names that we want to manually change 
# This can be used if we found errors in the data (for example, names were not scraped)
# This dataset is edited by hand, so it is stored in the input directory
name_mod = pd.read_excel(input_path / 'name_mod.xlsx')


## THESE THINGS SHOULD BE FUNCTION INPUTS
#This is how long the webdriver will wait while loading pages
delay = 5
# Data type: This is the type of data that we are using.
# The control group members only have one university url because they
# did not move. The lateral group members have two urls.
data_type = "lateral"

#This step sets the data (this will be a function parameter)
data = lateral


In [6]:
# Create a list to store the alternate name data
alt_names_data = []
# Create the alt-names dataset
alt_names_columns = ["first_mid_name", "last_name", "fm_names", "err_fm_names", "diff_last_name"]
# Add the names back into the dataframe of alternate names
df_alt_names = pd.DataFrame(columns = alt_names_columns)

In [4]:

#This loop goes through each name
for i in range(len(data)):
    #This section gets the professor's information from the dataframe 
    # Get variable values from the dataframe
    prof_id = data['ID'][i]
    mid_first_name = data['FirstName'][i]
    last_name = data['LastName'][i]
    full_name = mid_first_name + ' ' +  last_name
    #This line gets the school URLs from the dataframe
    if data_type == "lateral":
        school_url = [data['Short URL Origin'][i], data['Short URL Destination'][i]]
        school = data['Origin School'][i]
        new_school = data['Destination School'][i]
    elif data_type == "control":
        school_url = [data['Short URL Origin'][i]]
        school = data['Origin School'][i]

    # check_files checks if a file for the papers from the professor has already been created.
    #If a file has already been created for the professor, the loop moves onto the next name.
    if check_files(mid_first_name, last_name, current_data):
        print('File for ' + full_name + ' has already been created.')
        continue

    # Initilization
    # page_name = []
    # err_fm_names = []
    # df_sub = pd.DataFrame()
    print(mid_first_name)
    print(last_name)
    
    # Search by author to find potential alternative first and middle names:
    fm_names, err_fm_names = search_names(mid_first_name, last_name, school_url, driver, g_driver)


    # Create a list of values to append to the dataframe
    # We convert fm_names and err_fm_names to lists of strings during this step
    values_alt_names = [mid_first_name, last_name, fm_names, err_fm_names]
    dict_values_alt_names = dict(zip(alt_names_columns, values_alt_names))
    alt_names_data.append(dict_values_alt_names)

# Append all of the data to the ouput dataset and output to Excel
df_alt_names = df_alt_names.append(alt_names_data)
df_alt_names.to_excel(intr_path / "name_mod_test.xlsx")

Matthew
Adler


KeyboardInterrupt: 

In [39]:
alt_names_data

[{'first_mid_name': 'Matthew',
  'last_name': 'Adler',
  'fm_names': ['Matthew'],
  'err_fm_names': ['Matthew D.', 'Matthew H.'],
  'diff_last_name': ''},
 {'first_mid_name': 'Michael',
  'last_name': 'Frakes',
  'fm_names': ['Michael'],
  'err_fm_names': ['Michael D.'],
  'diff_last_name': ''},
 {'first_mid_name': 'Brandon',
  'last_name': 'Garrett',
  'fm_names': ['Brandon'],
  'err_fm_names': ['Brandon L.'],
  'diff_last_name': ''},
 {'first_mid_name': 'H.',
  'last_name': 'Powell',
  'fm_names': ['H.'],
  'err_fm_names': ['J. H.', 'Wm. H.'],
  'diff_last_name': ''},
 {'first_mid_name': 'Matthew',
  'last_name': 'Adler',
  'fm_names': ['Matthew'],
  'err_fm_names': ['Matthew D.', 'Matthew H.']},
 {'first_mid_name': 'Michael',
  'last_name': 'Frakes',
  'fm_names': ['Michael'],
  'err_fm_names': ['Michael D.']},
 {'first_mid_name': 'Brandon',
  'last_name': 'Garrett',
  'fm_names': ['Brandon'],
  'err_fm_names': ['Brandon L.']},
 {'first_mid_name': 'H.',
  'last_name': 'Powell',
  'f